In [1]:
import os
import re
import openai
import pinecone
from time import time, sleep

/Users/farisology/.pyenv/versions/3.11.0/envs/casiaenv/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
openai.api_key = "--"
pinecone.init(api_key = "--", environment = 'us-east1-gcp')
vdb = pinecone.Index("testbed")

def gpt3_embedding(content, engine='text-embedding-ada-002'):
    content = content.encode(encoding='ASCII',errors='ignore').decode()  # fix any UNICODE errors
    response = openai.Embedding.create(input=content,engine=engine)
    vector = response['data'][0]['embedding']  # this is a normal list
    return vector

In [3]:
a= "what plants to grow in Malaysia?"

vector = gpt3_embedding(a)
results = vdb.query(vector=vector, top_k=3, include_values=False, include_metadata=True, namespace="casia_kb")

In [9]:
def save_file(filepath, content):
    with open(filepath, 'w', encoding='utf-8') as outfile:
        outfile.write(content)

def chadgpt_completion(question, context):
    max_retry = 5
    retry = 0
    preample = """You are casia a wise botanist with vast knowledge in gardening and great servitude to human well-being. You will read the context passage -inclosed with tripple quotes- and reflect on the question then provide a specific beautiful answer using simple words with care and interest in the human sustainable way of living. Its your responsibility to answer based on what you know, if the context is not clear don't make up any answer and just reply humble that you don't know and maybe you can answer the question in the future.

Passage:"""
    while True:
        try:
            response = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages=[
                    {"role": "system", "content": f"{preample}"},
                    {"role": "assistant", "content": f"{context}"},
                    {"role": "user", "content": f"{question}"},
                ],
                temperature=0,
                top_p=1.0, max_tokens=500, frequency_penalty=0.0, presence_penalty=0.0,
            )
            text = response["choices"][0]["message"]["content"].strip()
            text = re.sub('[\r\n]+', '\n', text)
            text = re.sub('[\t ]+', ' ', text)
            filename = '%s_gpt3.txt' % time()
            if not os.path.exists('gpt3_logs'):
                os.makedirs('gpt3_logs')
            save_file('gpt3_logs/%s' % filename, preample +
                      '\n\n==========\n\n' + text)
            return text
        except Exception as oops:
            retry += 1
            if retry >= max_retry:
                return "GPT3.5 turbo error: %s" % oops
            print('Error communicating with OpenAI:', oops)
            sleep(1)

In [5]:
chadgpt_completion(a, '\n'.join([info["metadata"]["text"] for info in results["matches"]]))

"Malaysia has a tropical climate, which means that it is suitable for growing a wide variety of plants. Some plants that can be grown in Malaysia include:\n1. Orchids - Malaysia is known for its beautiful orchids, which thrive in the warm and humid climate.\n2. Banana trees - Bananas are a staple food in Malaysia, and banana trees are easy to grow in the tropical climate.\n3. Herbs - Many herbs, such as lemongrass, ginger, and turmeric, grow well in Malaysia and are commonly used in Malaysian cuisine.\n4. Tropical fruits - Malaysia is home to a wide variety of tropical fruits, including durian, rambutan, and mangosteen.\n5. Ferns - Ferns are a popular ornamental plant in Malaysia and can be grown both indoors and outdoors.\n6. Palm trees - Palm trees are a common sight in Malaysia and are used for both ornamental and commercial purposes.\n7. Pitcher plants - Malaysia is home to a variety of carnivorous plants, including the famous pitcher plant, which is native to the country.\n8. Bamb

## next question

In [10]:
a= "How to take care of orchids?"

vector = gpt3_embedding(a)
results = vdb.query(vector=vector, top_k=3, include_values=False, include_metadata=True, namespace="casia_kb")
chadgpt_completion(a, '\n'.join([info["metadata"]["text"] for info in results["matches"]]))

Error communicating with OpenAI: This model's maximum context length is 4097 tokens. However, your messages resulted in 6099 tokens. Please reduce the length of the messages.
Error communicating with OpenAI: This model's maximum context length is 4097 tokens. However, your messages resulted in 6099 tokens. Please reduce the length of the messages.
Error communicating with OpenAI: This model's maximum context length is 4097 tokens. However, your messages resulted in 6099 tokens. Please reduce the length of the messages.
Error communicating with OpenAI: This model's maximum context length is 4097 tokens. However, your messages resulted in 6099 tokens. Please reduce the length of the messages.


"GPT3.5 turbo error: This model's maximum context length is 4097 tokens. However, your messages resulted in 6099 tokens. Please reduce the length of the messages."

In [11]:
from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain
llm = OpenAI(openai_api_key="PROVIDE OPENAI_API_KEY HERE")


In [12]:
llm = OpenAI(openai_api_key= "--")

In [14]:
model = load_summarize_chain(llm=llm, chain_type="stuff")
model.run('\n'.join([info["metadata"]["text"] for info in results["matches"]]))

AttributeError: 'str' object has no attribute 'page_content'